In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
pd.__version__

---

- Basic building block: `pd.Timestamp`
- More building blocks: `pd.Period` and `freq`
- `ns`: nanoseconds
- Time series frequencies: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

In [ ]:
time_stamp = pd.Timestamp(datetime(2017, 1, 1))

In [ ]:
pd.Timestamp("2017-01-01") == time_stamp

In [ ]:
time_stamp

In [ ]:
type(time_stamp)

In [ ]:
time_stamp.year

In [ ]:
time_stamp.day_name()

In [ ]:
period = pd.Period("2017-01")
period

In [ ]:
period.asfreq("D")

In [ ]:
period.to_timestamp()

In [ ]:
period.to_timestamp().to_period("M")

In [ ]:
period, period + 2

In [ ]:
index = pd.date_range(start="2017-01-01", periods=12, freq="M")
index

In [ ]:
type(index)

Each date in `pd.DatetimeIndex` is a `pd.Timestamp`.

In [ ]:
index[0]

In [ ]:
type(index[0])

In [ ]:
index.to_period()

---

In [ ]:
google = pd.read_csv("data/google.csv")
google.info()

In [ ]:
google = google.dropna()
google.info()

In [ ]:
google.head()

In [ ]:
google.tail()

In [ ]:
google["Date"] = pd.to_datetime(google["Date"])
google.info()

In [ ]:
google.set_index("Date", inplace=True)
google.info()

## Partial string indexing

If you pass a string representing a year, pandas returns all dates within that year.

In [ ]:
# Deprecated
# google["2015"].info()

In [ ]:
google.loc["2015"].info()

In [ ]:
# Includes the end date
google["2015-03":"2016-02"].info()

In [ ]:
google.loc["2016-06-01", "Close"]

In [ ]:
google.asfreq("D").info()

In [ ]:
google.asfreq("D").head()

In [ ]:
google.asfreq("M").info()

In [ ]:
google.asfreq("M").head()

---

In [ ]:
google = pd.read_csv("data/google.csv", parse_dates=["Date"], index_col="Date")
google = google.dropna()

In [ ]:
google.info()

In [ ]:
google.head()

`.shift()`:

- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html
- Moving data between past and future
- Defaults to `periods=1` (1 period into the future)

In [ ]:
google["shifted"] = google["Close"].shift()
google.head()

In [ ]:
google["lagged"] = google["Close"].shift(periods=-1)
google.tail()

## Calculate one-period percent change

- $x_t / x_{t-1}$
- The factor by which you need to multiply the last price to get the current price

In [ ]:
google["change"] = google["Close"].div(google["shifted"])
google.head()

In [ ]:
# google["Close"] / google["shifted"]

In [ ]:
google["return"] = google["change"].sub(1).mul(100)
google.head()

`.diff()`: Difference in value for two adjacent periods.

In [ ]:
google["diff"] = google["Close"].diff()
google.head()

`.pct_change()`: Percent change for two adjacent periods.

In [ ]:
google["pct_change"] = google["Close"].pct_change().mul(100)
google.head()

---

In [ ]:
google = pd.read_csv("data/google.csv", parse_dates=["Date"], index_col="Date")
google = google.dropna()

In [ ]:
google.head()

In [ ]:
first_price = google["Close"].iloc[0]
first_price

In [ ]:
normalized = google["Close"].div(first_price).mul(100)
normalized

In [ ]:
prices = pd.read_csv("data/apple_google.csv", parse_dates=["Date"], index_col="Date")
prices = prices.dropna()

In [ ]:
prices.head()

In [ ]:
prices.shape

In [ ]:
first_prices = prices.iloc[0]
first_prices

In [ ]:
normalized = prices.div(first_prices).mul(100)
normalized.head()

In [ ]:
prices = pd.read_csv("data/apple_google.csv", parse_dates=["Date"], index_col="Date")
index = pd.read_csv("data/sp500.csv", parse_dates=["date"], index_col="date")

In [ ]:
prices = pd.concat([prices, index], axis=1).dropna()
prices.head()

In [ ]:
prices.info()

In [ ]:
normalized = prices.div(prices.iloc[0]).mul(100)
normalized.head()

Performance difference for each stock relative to the benchmark (`SP500`) in percentage points, subtract the normalized `SP500` from the normalized stock prices:

In [ ]:
diff = normalized[["AAPL", "GOOG"]].sub(normalized["SP500"], axis=0)
diff.head()

---

## Changing the frequency: resampling

- Upsampling: fill or interpolate missing data
- Downsampling: aggregate existing data

In [ ]:
dates = pd.date_range(start="2016", periods=4, freq="Q")

quarterly = pd.Series(data=range(1,5), index=dates)
quarterly

In [ ]:
monthly = quarterly.asfreq("M")
monthly

In [ ]:
monthly = monthly.to_frame("baseline")
monthly.head()

In [ ]:
# Forward fill
monthly["ffill"] = quarterly.asfreq("M", method="ffill")
# Backfill
monthly["bfill"] = quarterly.asfreq("M", method="bfill")
monthly["value"] = quarterly.asfreq("M", fill_value=0)

monthly

In [ ]:
dates = pd.date_range(start="2016", periods=12, freq="M")
dates

In [ ]:
quarterly

In [ ]:
quarterly.reindex(dates)

---

---